**version 4 :** 
trying to get better resault in main Algorithm
<br>
**changes :**
- keeps alive the 20% of the top (after 50 % progress)

**insights :**
<br>
- list of hyperparameter must tune that recognize:
 1. number of geneerations (time and fitness trade-off)
 <br>
 2. number of each populations
 <br>
 3. possibilities of having mutations
 <br>
 4. maximum number of depth that each tree can have
 <br>
 5. the way of crossovers (completely random or something diffrent)
 <br>
 6. change the power of mse in middle of way
 <br>
 7. the time that is better to keeps them alive 
 <br>
 8. the percentage of each gen must keeps alive
 <br>
</list>
- problems of domain in caclulates and infinity => cuz of cos or sin of infinity(?)


In [1]:
import random
import plotly.express as px
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error
import math
operators = ["+" , "-" , "*" , "/"  ] # can add pow to the set "^"
singleOperators = ["sin" , "cos"] # in sin & cos , right node is always NULL
x1 = 2
x2 = 5
listOfInputs = ["x1" , "x2"]
CoefficientsRagne = 10
allOperators = []
allOperators.extend(operators)
allOperators.extend(singleOperators)

In [4]:
class node ():
    def __init__(self , operator , rightnode = None , leftnode = None  , father = None):
        self.father = father
        self.leaf = False
        self.rightnode = rightnode
        self.leftnode = leftnode
        if operator in operators or operator in singleOperators  :
            self.operator = operator
        elif (type (operator) is int) or operator in listOfInputs : # this is the way we know the leaf and nodes
            self.leaf = True
            self.operator = operator 
        else :
            print("operator not assigned - wrong input")
        
    def assignOperator (self ):
        x = input()
        while (x not in operators and x not in singleOperators ) and not x.isnumeric():
            print("x is not true ! try again !")
            x = input()
        self.operator = x
        if x.isnumeric() : # clear the down nodes
            self.leaf = True
            self.leftnode = None
            self.rightnode = None
    def addNode (self , node):
        if self.leaf == False :
            if self.leftnode == None :
                self.leftnode = node
                node.father = self
                return True
            elif self.rightnode == None and self.operator not in singleOperators:
                self.rightnode = node
                node.father = self
                return True
            else:
                if self.leftnode.addNode(node) :
                    return True
                if self.operator not in singleOperators:
                    if self.rightnode.addNode(node) :
                        return True
            return False
        return False

In [3]:
test = node("")

operator not assigned - wrong input


In [4]:
test.assignOperator()

In [5]:
test

In [6]:
test.operator

'+'

In [5]:
class tree ():
    def __init__(self , root):
        self.numnerOfNodes = 1
        self.LeafList = []
        self.LeafList.append(root)
        if type( root ) is node :
            self.root = root
        else:
            print("try another time plz . ")
    def calculateFather(self,node):
        counter = 1 
        temp = node.father
        if temp != None :
            while temp.father != None :
                temp = temp.father
                counter = counter + 1
        return counter
    def addNodeInSurf (self , node ):
        MaxNumsOfFather = 1000
        nodeTemp = None
        for item in self.LeafList : 
            temp = self.calculateFather(item)
            if temp < MaxNumsOfFather :
                MaxNumsOfFather = temp
                nodeTemp = item
        if nodeTemp.leftnode == None :
            nodeTemp.leftnode = node
            node.father = nodeTemp
            self.LeafList.append(node)
            if nodeTemp.rightnode != None or nodeTemp.operator in singleOperators :
                self.LeafList.remove(nodeTemp)
        elif nodeTemp.rightnode == None and nodeTemp.operator not in singleOperators:
            nodeTemp.rightnode = node
            node.father = nodeTemp
            self.LeafList.append(node)
            if nodeTemp.leftnode != None : 
                self.LeafList.remove(nodeTemp)
        self.numnerOfNodes = self.numnerOfNodes + 1
    def addNodeIndepth (self , node):    
        if self.root.addNode(node) :
            self.LeafList.append(node)
            if node.father.operator in singleOperators :
                self.LeafList.remove(node.father)
        self.numnerOfNodes = self.numnerOfNodes + 1
    def isComplete(self):
        for item in self.LeafList :
            if item.leaf == False:
                return False
        return True
    def PostorderTraversal(self, root):
        res = []
        if root:
            res = self.PostorderTraversal(root.left)
            res = res + self.PostorderTraversal(root.right)
            res.append(root.data)
        return res

In [10]:
# test1 = node("-")
# test.addNode(test1)
# test.addNode(node(2))

In [60]:
def calculate (node , args):
    if node.leaf == True:
        if node.operator == "x1" :
           return args[0]
        if node.operator == "x2" :
           return args[1]
        return node.operator
    if node.operator == "+" :
        return calculate(node.leftnode , args) + calculate(node.rightnode , args)
    elif node.operator == "-" :
        return calculate(node.leftnode , args) - calculate(node.rightnode , args)
    elif node.operator == "/" :
        divded = calculate(node.rightnode , args)
        if divded == 0 :
            return float('inf')
        else :
            return calculate(node.leftnode , args) / divded
    elif node.operator == "*" :
        return calculate(node.leftnode , args) * calculate(node.rightnode , args)
    elif node.operator == "^" :
        return pow( calculate(node.leftnode , args) , calculate(node.rightnode , args) )
    elif node.operator == "sin" :
        return math.sin(calculate(node.leftnode , args))
    elif node.operator == "cos" :
        # print(node.leftnode.operator)
        return math.cos(calculate(node.leftnode , args))

In [7]:
def showtree(node) :
    if node.leaf == True:
        return  f"{node.operator}"
    if node.operator == "+" :
        return f"({showtree(node.leftnode)}) + ({showtree(node.rightnode)})"
    elif node.operator == "-" :
        return f"({showtree(node.leftnode)}) - ({showtree(node.rightnode)})"
    elif node.operator == "/" :
        return f"({showtree(node.leftnode)}) / ({showtree(node.rightnode)})"
    elif node.operator == "*" :
        return f"({showtree(node.leftnode)}) * ({showtree(node.rightnode)})"
    elif node.operator == "^" :
        return f"({showtree(node.leftnode)}) ^ ({showtree(node.rightnode)})"
    elif node.operator == "sin" :
        return f"sin({showtree(node.leftnode)})"
    elif node.operator == "cos" :
        return f"cos({showtree(node.leftnode)})"

In [10]:
def createRandomTree(max , Coefficients):
    numberOfDepth = random.randint(1 , max)
    # create the root
    firstOperator = random.randint(0 , len(allOperators) - 1)
    temptree = tree(node(allOperators[firstOperator]))
    # create the tree structure 
    for _ in range (0 , (2 ^ numberOfDepth) - 1) :
        firstOperator = random.randint(0 , len(allOperators) - 1)
        randomnode = temptree.LeafList[random.randint(1 , len(temptree.LeafList)) - 1]
        if temptree.calculateFather(randomnode) <= max :
            if firstOperator % 2 == 0 :
                if randomnode.rightnode == None and randomnode.operator not in singleOperators: 
                    randomnode.rightnode = node(allOperators[random.randint(0 , len(allOperators) - 1)] , father=randomnode)
                    temptree.numnerOfNodes = temptree.numnerOfNodes + 1
                    temptree.LeafList.append(randomnode.rightnode)
                elif randomnode.leftnode != None :
                    temptree.LeafList.remove(randomnode)
                else :
                    randomnode.leftnode = node(allOperators[random.randint(0 , len(allOperators) - 1)] , father=randomnode)
                    temptree.numnerOfNodes = temptree.numnerOfNodes + 1
                    temptree.LeafList.append(randomnode.leftnode)
                    temptree.LeafList.remove(randomnode)
            else:
                if randomnode.leftnode == None : 
                    randomnode.leftnode = node(allOperators[random.randint(0 , len(allOperators) - 1)] , father=randomnode)
                    temptree.numnerOfNodes = temptree.numnerOfNodes + 1
                    temptree.LeafList.append(randomnode.leftnode)
                elif randomnode.rightnode != None or randomnode.operator in singleOperators:
                    temptree.LeafList.remove(randomnode)
                else :
                    randomnode.rightnode = node(allOperators[random.randint(0 , len(allOperators) - 1)] , father=randomnode)
                    temptree.numnerOfNodes = temptree.numnerOfNodes + 1
                    temptree.LeafList.remove(randomnode)
                    temptree.LeafList.append(randomnode.rightnode)
            
    # fill the leafs
    templist = temptree.LeafList.copy()
    for lastnode in templist :
        firstOperator = random.randint(0 , len(allOperators) - 1)
        if lastnode.rightnode == None :
            if firstOperator % 2 == 0 :
                lastnode.rightnode = node( listOfInputs[random.randint(0 , len(listOfInputs) - 1)] , father=lastnode )
                temptree.numnerOfNodes = temptree.numnerOfNodes + 1
                temptree.LeafList.append(lastnode.rightnode)
            else :
                lastnode.rightnode = node( random.randint(-1 * Coefficients , Coefficients ) , father=lastnode )
                temptree.numnerOfNodes = temptree.numnerOfNodes + 1
                temptree.LeafList.append(lastnode.rightnode)
            if lastnode.leftnode != None :
                temptree.LeafList.remove(lastnode)
                #templist.remove(lastnode)
        if lastnode.leftnode == None :
            if firstOperator % 2 == 1 :
                lastnode.leftnode = node( listOfInputs[random.randint(0 , len(listOfInputs) - 1)] , father=lastnode )
                temptree.numnerOfNodes = temptree.numnerOfNodes + 1
                temptree.LeafList.append(lastnode.leftnode)
            else :
                lastnode.leftnode = node( random.randint (-1 * Coefficients , Coefficients ) , father=lastnode )
                temptree.numnerOfNodes = temptree.numnerOfNodes + 1
                temptree.LeafList.append(lastnode.leftnode)
            if lastnode.rightnode != None :
                temptree.LeafList.remove(lastnode)
                #templist.remove(lastnode)
    return temptree
        
        

**test the add in surf**

In [114]:
testtree2= tree(test)

In [115]:
testtree2.LeafList

In [116]:
testtree2.addNodeInSurf(node("+"))

In [117]:
testtree2.addNodeInSurf(node("+"))

In [118]:
testtree2.addNodeInSurf(node("-"))

In [119]:
testtree2.addNodeInSurf(node("-"))

**test add in depth**

In [120]:
testtree2.addNodeIndepth(node("-"))

In [121]:
testtree2.addNodeIndepth(node("-"))

**test the calculate**

In [11]:
testtree = tree(node("sin"))

In [12]:
#simple test with 
testtree.addNodeInSurf(node("/"))
testtree.addNodeInSurf(node("*"))
testtree.addNodeInSurf(node(x1))
testtree.addNodeInSurf(node(x2))
testtree.addNodeInSurf(node(x1))
testtree.addNodeInSurf(node(x2))
print( calculate(testtree.root , (2 , 5)) )

-0.9589242746631385


**test the showtree**

In [13]:
print(showtree(testtree.root))

sin(((5) * (2)) / (2))


**test the random creation**

In [20]:
testRandom1 =  createRandomTree(10 , 5)

In [21]:
print(showtree(testRandom1.root))

((-4) - (((2) / (x1)) / ((cos(x1)) * (x1)))) * (sin(cos(cos((3) + (x1)))))


In [22]:
print(calculate( testRandom1.root , (3 , 6)))

-2.0481313613372


In [23]:
testRandom1.numnerOfNodes

19

**the main function**

In [24]:
def calculateValue(inputValues , root):
    predicted = []
    for tupleIn in inputValues:
        predicted.append( calculate(root , tupleIn))
    return predicted

In [25]:
def RandomSelection(mses , population):
    total = []
    cumsum = 0 
    for i in mses :
        cumsum = cumsum + i
        total.append(cumsum)
    rnd = random.random() * cumsum
    for i, uptotal in enumerate(total):
        if rnd < uptotal:
            return population[i]
    return population[-1] # if something goes wrong !

In [26]:
import copy
def iterate ( node , numberOfrotate , visited):
    # temp = copy.deepcopy(node)
    # print(numberOfrotate)
    visited = []
    counter = 0 
    while counter != numberOfrotate:
        if node.rightnode != None and node.rightnode not in visited:
            counter = counter + 1
            visited.append(node.rightnode)
            node = node.rightnode
        elif node.leftnode != None and node.leftnode not in visited :
            counter = counter + 1
            visited.append(node.leftnode)
            node = node.leftnode
        else :
            if node.father != None :
                node = node.father
            else :
                node = node.leftnode
                counter = counter + 1
    return node   

In [24]:
testtree = tree(node("-"))
testtree.root.leftnode = node("+" , father=testtree.root)
testtree.root.rightnode = node(4 , father=testtree.root)
testtree.root.leftnode.leftnode = node(4 , father=testtree.root.leftnode)
testtree.root.leftnode.rightnode = node("+" , father = testtree.root.leftnode)
testtree.root.leftnode.rightnode.leftnode = node("-" , father=testtree.root.leftnode.rightnode)
testtree.root.leftnode.rightnode.leftnode.leftnode = node("x1" , father=testtree.root.leftnode.rightnode.leftnode)
testtree.root.leftnode.rightnode.leftnode.rightnode = node(7, father=testtree.root.leftnode.rightnode.leftnode)
testtree.root.leftnode.rightnode.rightnode = node("-" , father=testtree.root.leftnode.rightnode)
testtree.root.leftnode.rightnode.rightnode.leftnode = node("x2" , father=testtree.root.leftnode.rightnode.rightnode)
testtree.root.leftnode.rightnode.rightnode.rightnode = node(5 , father=testtree.root.leftnode.rightnode.rightnode)
testtree.numnerOfNodes = 11
print(showtree(testtree.root))

((4) + (((x1) - (7)) + ((x2) - (5)))) ^ (4)


In [29]:
Mutation(testtree)

In [30]:
print(showtree(testtree.root))

((x1) + (((8) - (7)) ^ ((x2) - (5)))) ^ (4)


**test the iterate**


In [31]:
testtree2 = tree(node("+"))
testtree2.addNodeInSurf(node(x1))
testtree2.addNodeInSurf(node(x2))
print(showtree(testtree2.root))
print(iterate(testtree2.root , 1 , [] ).operator)

(2) + (5)
5


In [27]:
import copy

def CrossOver (x , y) :
    maxOfNodes = 0 
    if x.numnerOfNodes > y.numnerOfNodes :
        maxOfNodes = y.numnerOfNodes
    else :
        maxOfNodes = x.numnerOfNodes
    rnd = random.randint( 1 , maxOfNodes - 1 )
    # print(rnd)
    
    temptree = copy.deepcopy(x)
    temptree1 = copy.deepcopy(y)
    
    firstNode = iterate(temptree.root , rnd , [])
    secondNode = iterate(temptree1.root , rnd , [])
    firstFather =  firstNode.father 
    secondFather =  secondNode.father 
    
    temp = copy.deepcopy(firstNode)
    # print(temp.operator)
    temp1 = copy.deepcopy(secondNode)
    # print(temp1.operator)
    
    
    if firstFather.leftnode == firstNode :
        firstFather.leftnode = temp1
    else :
        firstFather.rightnode = temp1
    
    if secondFather.leftnode == secondNode :
        secondFather.leftnode = temp
    else :
        secondFather.rightnode = temp
    
    temp1.father = firstFather
    temp.father = secondFather
    
    del firstNode
    del secondNode
    
    return temptree , temptree1

**test the CrossOver**

In [28]:
testtree = tree(node("-")) # check the cross over
testtree.addNodeInSurf(node(x2))
testtree.addNodeInSurf(node(x1))
print(showtree(testtree.root))
testtree2 = tree(node("+"))
testtree2.addNodeInSurf(node(x1))
testtree2.addNodeInSurf(node(x2))
print(showtree(testtree2.root))
temptest , temptest1 = CrossOver(testtree , testtree2)
print(showtree(temptest.root))
print(showtree(temptest1.root))


(5) - (2)
(2) + (5)
(5) - (5)
(2) + (2)


In [29]:
def Mutation (tree) :
    rnd = random.randint( 1 , tree.numnerOfNodes - 1 )
    #print(showtree(tree.root))
    changeNode = iterate(  tree.root , rnd , [] )
    if changeNode.operator in allOperators :
        changeNode.operator = allOperators[random.randint(0 , len(allOperators) - 1)]        
        if changeNode.operator in singleOperators :
            changeNode.rightnode = None
        elif changeNode.rightnode == None :
            changeNode.rightnode = node( listOfInputs[random.randint(0 , len(listOfInputs) - 1)] , father=changeNode )           
    else:
        if rnd % 2 == 0 :
            changeNode.operator = listOfInputs[random.randint(0 , len(listOfInputs) - 1)]
        else :
            changeNode.operator = random.randint (-1 * CoefficientsRagne , CoefficientsRagne )

In [30]:
AllData = []

In [121]:
def general_algorithm(NumOfFirstPop , maxOfDepth , numsOfGenereations , inputValue , trueValue , AllMses):
    # first create a population 
    population = []
    for _ in range(0 , NumOfFirstPop):
        population.append(createRandomTree(maxOfDepth , CoefficientsRagne))
    # main loop
    AllMses = []
    counter = 0 
    while counter != numsOfGenereations :
        # fitness values
        mses = []
        for i in range(0 , NumOfFirstPop):
            mses.append(mean_squared_error(trueValue , calculateValue(inputValue , population[i].root ) ))    
        print(mses)
        # keep bests alive!
        newPopulation = []
        if counter >= numsOfGenereations / 2 :
            tempList = [X for _ , X in sorted(zip( mses , population ) , key= lambda elem : elem[0])]
            for i in range(math.floor( len(population) * 0.2)) :
                newPopulation.append(tempList[i])
        # change the mse up to it is negetive fitness
        AllMses.append(mses)
        MaxOfmses = max( mses )
        for i in range(0 , NumOfFirstPop):
            mses[i] = mses[i] * -1 + MaxOfmses
        # select randomly with their fitness
        for _ in range(math.floor( (len(population) - len(newPopulation)) / 2)):
            # random select each tree for crossover
            firstNode = RandomSelection(mses , population)    
            secondNode = RandomSelection(mses , population)
            firstNewTree , secondNewTree = CrossOver(firstNode , secondNode)
            # mutations !
            rnd = random.randint(1 , 100)
            if rnd % 9 == 0 : 
                Mutation(firstNewTree) 
            rnd = random.randint(1 , 10)
            if rnd % 9 == 1 : 
                Mutation(secondNewTree) 
            newPopulation.append(firstNewTree)
            newPopulation.append(secondNewTree)
        population = newPopulation
        counter = counter + 1
    # return the best
    indexOfBest = 0 
    MaxOfmses = 0  
    for index , mse in enumerate (mses) :
        if mse > MaxOfmses : 
            indexOfBest = index 
            MaxOfmses = mse
    print(-1 * (max(mses) - MaxOfmses))
    return  AllMses

**create the test for the Main Algorithm**

In [49]:
testMain1 = createRandomTree(9 , CoefficientsRagne )
inputValue = [(10 , 20) , (20 , 30 ) , ( 40 , 50 ) , (60 , 70 ) , (90 , 80)]
trueValues = []
trueValues = calculateValue(inputValue , testMain1.root)
print(showtree(testMain1.root))

((-6) / (x1)) * ((7) + (x2))


In [50]:
trueValues

[-16.2, -11.1, -8.549999999999999, -7.7, -5.8]

In [123]:
AllData = general_algorithm(10 , 10 , 10 , inputValue , trueValues , AllData) 

[255.16164679000912, 96.12621138286232, 118.33681082378948, 376273.4192707446, 111.06787792592881, 99.87735797660422, 111.73901194182929, 120.78272102950811, 100.94195767186604, 106.4532787878585]
[118.96740924963596, 99.87735797660422, 117.7004109330234, 120.56618599020246, 122.29917075073608, 119.10091958473092, 137.20832182985365, 98.81638751175821, 226.29327953552584, 96.12621138286232]
[222.00127967269464, 98.81638751175821, 22.828189764480282, 131.06940175229772, 108.09616279487845, 256.69974577353116, 117.46602293809683, 80.45515165934204, 2667.5658498724965, 107.68489242090625]
[3188.409596871982, 117.46602293809683, 942.4713975731007, 137.20832182985365, 115.82541879946814, 105.78398740295408, 108.21869811382749, 100.94195767186604, 472.80186927768, 107.49547399842845]
[119.82451191696127, 117.46602293809683, 256.69974577353116, 3938.085507740571, 139.73244822081406, 120.45872677894329, 472.80186927768, 100.94195767186604, 94.53865200391085, 115.82541879946814]
[522.1987215955

In [109]:
np.shape(AllData)

(10, 10)

In [44]:
df = pd.DataFrame(AllData)

In [45]:
df

,0,1,2,3,4,5,6,7,8,9
0,3.197774e+05,3.196673e+05,0.000000e+00,3.195210e+05,3.190461e+05,2.053391e+05,3.197108e+05,3.193683e+05,3.197767e+05,3.225784e+05
1,9.505959e+06,9.506233e+06,9.506004e+06,0.000000e+00,9.506127e+06,9.506082e+06,9.509035e+06,9.505950e+06,9.504082e+06,9.500896e+06
2,5.856415e+06,5.856330e+06,5.855970e+06,5.856591e+06,5.854547e+06,5.859500e+06,5.859800e+06,5.856592e+06,0.000000e+00,5.856517e+06
3,1.905540e+06,1.905540e+06,0.000000e+00,1.905046e+06,1.902328e+06,1.900367e+06,1.891895e+06,1.902328e+06,1.902339e+06,1.905678e+06
4,5.295708e+03,8.562507e+02,5.315668e+03,0.000000e+00,5.417615e+03,5.272331e+03,5.272331e+03,4.160803e+03,5.394151e+03,2.060538e+03
5,1.564912e+03,3.293303e+03,3.350190e+03,2.717868e+03,2.987922e+03,3.350190e+03,3.235170e+03,0.000000e+00,3.333613e+03,3.333613e+03
6,1.880484e+05,1.869369e+05,1.881702e+05,0.000000e+00,1.880718e+05,1.881868e+05,1.860651e+05,1.872314e+05,1.880230e+05,1.881937e+05
7,1.489339e+03,2.006641e+03,8.717358e+02,8.717358e+02,1.957886e+03,2.121661e+03,2.128548e+03,2.128548e+03,2.105083e+03,0.000000e+00
8,5.754347e+02,6.323220e+02,0.000000e+00,0.000000e+00,5.173024e+02,4.685474e+02,6.323220e+02,5.754347e+02,6.157446e+02,6.323220e+02
9,1.751453e+02,0.000000e+00,2.235424e+02,5.288056e+01,2.166551e+02,1.016355e+02,2.000778e+02,2.204623e+02,5.288056e+01,1.597678e+02


In [52]:
px.scatter(df    , y = df.columns).show()